# Extract information from Chinese catalogue card xmls

In [1]:
from __future__ import annotations
import sys
if '../' not in sys.path:
    sys.path.append('../')
import os
import glob
import re 
import xml.etree.ElementTree as ET
from IPython.display import Image
from numpy import object_
import pandas as pd
import requests
import pickle
import matplotlib.pyplot as plt
from z3950.PyZ3950 import zoom
from z3950.Marc.marc_tools import Record
from tqdm import tqdm

## Parsing xmls

In [2]:
p5_local = "..\\data\\raw\\chinese"

In [3]:
attempts = 0
# page_xml_loc = os.path.join(p5_root, "page")
page_xml_loc = os.path.join(p5_local, "1016992")
while attempts < 3:
    xmls = glob.glob(os.path.join(p5_local, "1016992", "*.pxml"))
    if len(xmls) > 0:
        break
    else:
        attempts += 1
        continue
else:
    raise IOError(f"Failed to connect to {page_xml_loc}")
    
xmls

['..\\data\\raw\\chinese\\1016992\\28445611084.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445611154.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445611354.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445611384.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445611494.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445611564.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445611664.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445611674.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445611824.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445611874.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445611984.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445612034.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445612134.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445612334.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445612404.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445612564.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445612664.pxml',
 '..\\data\\raw\\chinese\\1016992\\28445612754.pxml',
 '..\\data\\raw\\chinese\\10

In [4]:
xmlroots = []
print(f"\nGetting xml roots from {page_xml_loc}")
for file in tqdm(xmls):
    fileName = os.fsdecode(file)
    attempts = 0
    while attempts < 3:
        try:
            tree = ET.parse(fileName)
            break
        except FileNotFoundError:
            attempts += 1
            continue
    else:
        raise FileNotFoundError(f"Failed to connect to: {fileName}")
    root = tree.getroot()
    xmlroots.append(root)


Getting xml roots from ..\data\raw\chinese\1016992


100%|███████████████████████████████████████████████████████████████████████████████| 401/401 [00:00<00:00, 921.52it/s]


In [5]:
len(xmlroots)

401

In [6]:
def extractLines(root: xml.etree.ElementTree.Element):
    lines = []

    textRegions = [x for x in root[1] if len(x) > 2]  # Empty Text Regions Removed

    for textRegion in textRegions:
        textLines = textRegion[1:-1]  # Skip coordinate data in first child
        for textLine in textLines:
            lines.append(textLine[-1][0].text)  # Text equivalent for line
    return lines

def extractLinesForVol(vol: list[xml.etree.ElementTree.Element]):
    allLines = []
    for root in tqdm(vol):
        rootLines = extractLines(root)
        allLines.append(rootLines) 
    return allLines

In [7]:
def extractLinesForVol(vol: list[xml.etree.ElementTree.Element]):
    allLines = []
    for root in tqdm(vol):
        rootLines = extractLines(root)
        allLines.append(rootLines) 
    return allLines

In [8]:
caps_regex = re.compile("[A-Z][A-Z][A-Z]+")
c_num_regex = re.compile("C\.[0-9]")  # C number title references
i_num_regex = re.compile("I[ABC]\.\s[0-9]")  # I number title references
date_regex = re.compile("1[45][0-9][0-9]")  # Date format regexes (specific to this volume)
smark_regex = re.compile("[0-9]{1,5}[\s\.]{1,2}[\w]{1,3}[\s\.]{1,2}[\w0-9]{1,5}")
author_regex = re.compile("[A-Z]+[\s]+\([A-Z][a-z]+\)")
isbn_regex = re.compile("ISBN\s[0-9\-\s]+")

In [9]:
cards = extractLinesForVol(xmlroots)

100%|████████████████████████████████████████████████████████████████████████████| 401/401 [00:00<00:00, 100221.42it/s]


In [10]:
cards_df_v0 = pd.DataFrame(
    data={
        "xml": [os.path.basename(x) for x in xmls],
        "lines": cards,
        "dummy": [None for x in cards]
    }
)

In [11]:
def find_author(lines, dummy):
    author, title = None, None
    
    for i, l in enumerate(lines):
        if author_regex.search(l):  # look for an author format match
            author = l
            break
    
    if author:
        if i >= 2:  # author is after the second line (where we expect the title)
            title = " ".join(lines[1:i])
        elif i == 1:  # author is the second line
            title = lines[2]
    else:
        title = lines[1]  # default to the title being the second line
        
    return title, author

In [12]:
def isbn_search(x):
    if type(x) is not list:
        raise TypeError(f"List expected not {type(x)}")
    res = isbn_regex.search("".join(x))
    if res:
        return res.group().replace("-", "").replace(" ", "").lstrip("ISBN")
    else:
        return None

In [13]:
def shelfmark_search(x):
    if smark_regex.search(x[0]):
        result = smark_regex.search(x[0]).group()
    elif smark_regex.search(x[1]):
        result = smark_regex.search(x[1]).group()
    else:
        return None
    return result.replace(" ", "")

In [14]:
cards_df_v0["shelfmark"] = cards_df_v0["lines"].transform(shelfmark_search)
t_a = cards_df_v0.loc[:,('lines', 'dummy')].transform(lambda x: find_author(x[0], x[1]), axis=1).rename(columns={"lines":"title", "dummy":"author"})
cards_df = cards_df_v0.drop(columns="dummy").join(t_a)
cards_df["ISBN"] = cards_df["lines"].transform(lambda x:isbn_search(x))

In [15]:
cards_df.dropna()

,xml,lines,shelfmark,title,author,ISBN
39,28445619584.pxml,"[15592.c.22, FEI HSIAO-T'UNG: THE DILEMMA OF A...",15592.c.22,FEI HSIAO-T'UNG: THE DILEMMA OF A CHINESE INTE...,FEI (Xiaotong),0873321413
56,28445627074.pxml,"[CHI.1992.a.363, HUA (Sha), Fang da lu de <zou...",1992.a.363,Fang da lu de <zou si wang guo>,HUA (Sha),9623411111
104,28448804803.pxml,"[CHI.1993.a.85, HUANG (Weihong), Fei lan ao me...",1993.a.85,Fei lan ao men,HUANG (Weihong),9627602035
149,28780462740.pxml,"[CHI.1987.a.206, CHEN (Rongzhao), Fang zhong y...",1987.a.206,Fang zhong yan yan jiu,CHEN (Rongzhao),9620105218
156,28780471530.pxml,"[CHI.1986.a.1642, FAN (Chengda), Fan Chengda s...",1986.a.1642,Fan Chengda shi xuan,FAN (Chengda),9620403517
212,28961232382.pxml,"[CHI.1993.b.2, FANG (Renrong), Fang song ben x...",1993.b.2,Fang song ben xin ding xu zhi,FANG (Renrong),7805683948
306,28990275651.pxml,"[CHI.1993.a.223, WANG (Qishu), Fei hong tang y...",1993.a.223,Fei hong tang yin pu,WANG (Qishu),7532511448
361,29033823916.pxml,"[CHI.1993.a.68, WANG (Zihui), Fang tang cai di...",1993.a.68,Fang tang cai dian,WANG (Zihui),9627281373


In [16]:
cards_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   xml        401 non-null    object
 1   lines      401 non-null    object
 2   shelfmark  401 non-null    object
 3   title      401 non-null    object
 4   author     280 non-null    object
 5   ISBN       17 non-null     object
dtypes: object(6)
memory usage: 18.9+ KB


In [17]:
def OCLC_query(title="", author="", ISBN=None):
    # TODO Connection currently only handles 450 results at once before closing, extend or allow to reopen after 450
    conn = zoom.Connection(
        host='zcat.oclc.org',
        port=210,
        user='100270667',
        password='oclccat',
        databaseName='OLUCWorldCat',
        preferredRecordSyntax='USMARC'
    )
    res = None
    
    if ISBN:
        q = f'bn="{ISBN}"'
        print(q)
        query = zoom.Query(typ="CCL", query=q)
        res = conn.search(query)
        
    if not res:
        q = f'ti="{title}" and au="{author}"'
        print(q)
        query = zoom.Query(typ="CCL", query=q)
        res = conn.search(query)
        

    if res:
        try:
            res_dict = {i: r for i, r in enumerate(res)}
            conn.close()
            return res_dict
        except zoom.Bib1Err:
            print("Bib1Err")
            return "Bib1Err"
        except zoom.ProtocolError:
            print("ProtocolError - likely Diag")
            return "ProtocolError"
        except TypeError:
            print("Diag error")
            return None
    else:
        conn.close()
        return None

In [18]:
res = cards_df.apply(lambda x: OCLC_query(x['title'], x['author'], x['ISBN']),axis=1)

ti="FENG LING DU" and au="DUANMU (Hongliang)"
('Extracting', 16, 'starting at', 0)
ti="FENG LEI" and au="CHEN (Zhenke)"
ti="FENG KANG HOU SHU HUA YIN JI" and au="FENG (Kangho)"
ti="FENG JIN YI HE QUAN HUI LU" and au="LAO (Naixuan)"
('Extracting', 18, 'starting at', 0)
ti="FENG JIAN ZHU YI DE SHENG CHAN FANG SHI" and au="ZHANG (Yu)"
Bib1Err
ti="FENG JIAN SHE HUI" and au="SHI (Xing)"
Bib1Err
ti="FENG JIAN LUN" and au="ZHOU (Yiliang)"
('Extracting', 3, 'starting at', 0)
ti="Feng jian mi xin shi zen me hui shi?" and au="LI (Chuanming)"
('Extracting', 2, 'starting at', 0)
ti="FENG JIAN GUI ZU DA DI ZHU DE DIAN XING" and au="None"
ti="FENG JIA DA XUE GAI KUANG" and au="None"
ti="FENG JI CAI XUAN JI" and au="FENG (Jicai)"
('Extracting', 20, 'starting at', 0)
('Extracting', 20, 'starting at', 20)
ti="FENG JI CAI ZHONG DUAN PIAN XIAO SHUO JI" and au="FENG (Jicai)"
('Extracting', 3, 'starting at', 0)
ti="FENG HUO XI YANG HONG" and au="YI (Junzuo)"
('Extracting', 8, 'starting at', 0)
ti="FENG HUA

Unable to parse character 0x20 in g0=51 g1=69


ti="Fen lei gu jin shi hua" and au="XU (Qile)"
('Extracting', 4, 'starting at', 0)
ti="Fei you cun di" and au="SHEN (Congwen)"
('Extracting', 20, 'starting at', 0)
('Extracting', 20, 'starting at', 20)
ti="FEI XIN SU MIAO TOU XIANG XUAN" and au="FEIXIN (Nigula)"
ti="Fei xiang Ming wang xing de ren" and au="YE (Yonglie)"
('Extracting', 5, 'starting at', 0)
ti="FEI XIA SHEN DAO" and au="ZHOU (Tianxing)"
('Extracting', 1, 'starting at', 0)
ti="FEI XI JI" and au="SHI (He)"
('Extracting', 2, 'starting at', 0)
('Extracting', 2, 'starting at', 2)
('Extracting', 2, 'starting at', 4)
('Extracting', 2, 'starting at', 6)
('Extracting', 2, 'starting at', 8)
('Extracting', 2, 'starting at', 10)
('Extracting', 2, 'starting at', 12)
('Extracting', 2, 'starting at', 14)
('Extracting', 2, 'starting at', 16)
('Extracting', 2, 'starting at', 18)
('Extracting', 2, 'starting at', 20)
('Extracting', 2, 'starting at', 22)
('Extracting', 2, 'starting at', 24)
('Extracting', 2, 'starting at', 26)
('Extracting'

QuerySyntaxError: Parse p_error LexToken(RELOP,'=',1,2)

In [ ]:
# pickle.dump(res, open("res.p", 'wb'))

In [ ]:
# res = pickle.load(open("res.p", "rb"))  # pickled it because it takes about 10 mins to run the query

In [ ]:
all_records = res[res.str.contains("Error").isna()].dropna().apply(lambda x:[v for v in x.values()]).sum()

In [ ]:
cards_df['worldcat_result'] = res

In [ ]:
print(cards_df.loc[0, "worldcat_result"][0])

In [ ]:
sample_match_indices = [
    [0, 0, 1, 3],
    [1, 0, 1, 2],
    [6, 0, 1, 2]
]

In [ ]:
cards_df.dropna(subset="worldcat_result").iloc[6]

In [ ]:
idx

In [ ]:
cards_df.dropna(subset="worldcat_result").iloc[1, -4]

In [ ]:
for idx in sample_match_indices:
    print(cards_df.dropna(subset="worldcat_result").iloc[idx[0], 4])
    sample = cards_df.dropna(subset="worldcat_result").iloc[idx[0], -4]
    f001s = [print(x.get_fields("001")[0]) for x in [list(sample.values())[x] for x in idx[1:]]]

In [ ]:
sample1 = cards_df.dropna(subset="worldcat_result").iloc[6, -4]
[print(x.get_fields("001")[0]) for x in sample1.values()]

### Result selection

Once the results are back from Worldcat we need to pick which to use if multiple have been returned. For Urdu this is less of an issue as there are fewer results returned. For Chinese there are in some instances upwards of 500 records so we do need to select.

Victoria Morris suggested:
- Leader position 17
    - Encoding level, avoid 3/5/7
- 016 where \$2 subfield contains Uk (existing BL holding)
    - 16 is National bibliographic agency control number, $2Uk indicates it's the BL
- 040 
    - Cataloguing source (if "OCLC" appears lots of times then that's good)
- 042
    - Authentication code (quality control process)
- 100/110 and 245 (Searching on these)
    - Author/Corporate Author and Title
- 264/260
    - Old/new format publication information - 260 preferable to 264
- 300
    - Physical description
- 6XX 
    - Subject access
- 880 fields (linked to 100/245 or otherwise)
    - Linked to other fields, indicating original text formats

In [ ]:
def n_str(record, string):
    if type(record) is Record:
        f040 = record.get_fields("040")[0].text().split(" ")
        oclc_occurences = sum([1 for x in f040 if string in x])
        return oclc_occurences
    else:
        return None

    
def len_record(record):
    return len(record.get_fields())


def apply_n_str(x, string):
    if type(x) is dict:
        return [n_str(v, string) for v in x.values()]
    else:
        return None

    
def apply_len_record(x):
    if type(x) is dict:
        return [len_record(v) for v in x.values()]
    else:
        return None
    
    
def blx_holdings_exist(record):
    no_holdings = "NO HOLDINGS IN BLX" in record.get_fields("948")[0].text()
    if record.get_fields("016"):
        bl_system_number = "$2Uk" not in record.get_fields("016")[0].__str__()
    if no_holdings or bl_system_number:
        return 0
    else:
        return 1

In [ ]:
len(all_records)

In [ ]:
blx_holdings = [blx_holdings_exist(r) for r in all_records]

In [ ]:
sum(blx_holdings)

In [ ]:
oclc_count = res.apply(apply_n_str, string="OCLC")
bl_count = res.apply(apply_n_str, string="BLX")
record_len = res.apply(apply_len_record)

In [ ]:
cards_df["040_oclc_count"] = oclc_count
cards_df["040_bl_count"] = bl_count
cards_df["record_len"] = record_len

In [ ]:
sum(cards_df['040_bl_count'].dropna().sum())

In [ ]:
record_len.dropna().apply(lambda x: len(x)).describe()

In [ ]:
record_len.dropna().apply(lambda x: len(x)).hist(bins=40)

In [ ]:
cards_df.dropna(subset=["040_oclc_count"])

In [ ]:
oclc_count_df = pd.DataFrame(
    data={
        "040_oclc_count": cards_df['040_oclc_count'].dropna().sum(),
        "record_len": cards_df['record_len'].dropna().sum(),
    }
)

In [ ]:
oclc_count_df.shape

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.scatter("record_len", "040_oclc_count", data=oclc_count_df)
ax.set_xlabel("Fields in record", fontsize='x-large')
ax.set_ylabel("Occurences of 'OCLC' in MARC field 040", fontsize='x-large')
ax.set_title("Relationship between number of occurences of 'OCLC'\nin MARC field 040 in a record and total fields in a record", fontsize="x-large")
ax.tick_params(labelsize="x-large")

In [ ]:
# fig.savefig("..//reports//figures//OCLC_record_length.png", dpi=300, bbox_inches="tight")